## Search a STAC catalog

In [1]:
api_url = 'https://earth-search.aws.element84.com/v1'

In [2]:
from pystac_client import Client

client = Client.open(api_url)

In [3]:
collection = 'sentinel-2-l2a'  # Sentinel-2, Level 2A

In [4]:
from shapely.geometry import Point
point = Point(27.95, 36.20)  # Rhodes

In [5]:
search = client.search(
    collections=[collection],
    intersects=point,
)

In [6]:
search.matched()

513

### Exercise: searching satellite scenes with a time filter

Add a time filter to the search in order to select the only scenes recorded between 1 July and 31 August. You can find the input argument and the required syntax in the documentation of `client.search` (which you can access from Python or [online](https://pystac-client.readthedocs.io/en/stable/api.html#pystac_client.Client.search)). How many scenes do now match our search?

In [7]:
search = client.search(
    collections=[collection],
    intersects=point,
    datetime='2023-07-01/2023-08-31',
)

In [8]:
search.matched()

12

In [9]:
items = search.item_collection()

In [10]:
len(items)

12

In [11]:
for item in items:
    print(item)

<Item id=S2A_35SNA_20230827_0_L2A>
<Item id=S2B_35SNA_20230822_0_L2A>
<Item id=S2A_35SNA_20230817_0_L2A>
<Item id=S2B_35SNA_20230812_0_L2A>
<Item id=S2A_35SNA_20230807_0_L2A>
<Item id=S2B_35SNA_20230802_0_L2A>
<Item id=S2A_35SNA_20230728_0_L2A>
<Item id=S2B_35SNA_20230723_0_L2A>
<Item id=S2A_35SNA_20230718_0_L2A>
<Item id=S2B_35SNA_20230713_0_L2A>
<Item id=S2A_35SNA_20230708_0_L2A>
<Item id=S2B_35SNA_20230703_0_L2A>


In [12]:
item = items[-1]
print(item.datetime)
print(item.geometry)
print(item.properties)

2023-07-03 09:00:20.838000+00:00
{'type': 'Polygon', 'coordinates': [[[27.300972793493344, 37.046192287628024], [28.234426643135546, 37.04015200857309], [28.21878905911668, 36.05053734221328], [27.0215827618405, 36.056731037977386], [27.244605547808312, 36.843576366095995], [27.300972793493344, 37.046192287628024]]]}
{'created': '2023-07-03T16:57:15.486Z', 'platform': 'sentinel-2b', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 1.351987, 'proj:epsg': 32635, 'mgrs:utm_zone': 35, 'mgrs:latitude_band': 'S', 'mgrs:grid_square': 'NA', 'grid:code': 'MGRS-35SNA', 'view:sun_azimuth': 125.524648568983, 'view:sun_elevation': 69.1288988657908, 's2:degraded_msi_data_percentage': 0.0167, 's2:nodata_pixel_percentage': 12.999658, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 0.094058, 's2:cloud_shadow_percentage': 0.088758, 's2:vegetation_percentage': 7.767729, 's2:not_vegetated_percentage': 16.054803, 's2:water_percentage': 74.22775, 's2:uncl

### Exercise: searching satellite scenes using metadata filters

Let's add a filter on the cloud cover to select the only scenes with less than 1% cloud coverage. How many scenes do now match our search?

Hint: generic metadata filters can be implemented via the `query` input argument of `client.search`, which requires the following syntax (see [docs](https://pystac-client.readthedocs.io/en/stable/usage.html#query-extension)): `query=['<property><operator><value>']`.

In [42]:
search = client.search(
    collections=[collection],
    intersects=point,
    datetime='2023-07-01/2023-08-31',
    query=['eo:cloud_cover<1']
)

In [43]:
print(search.matched())

11


In [44]:
items = search.item_collection()

In [16]:
items.save_object('rhodes_sentinel-2.json')

## Access the assets

In [17]:
assets = items[0].assets  # first item's asset dictionary
print(assets.keys())

dict_keys(['aot', 'blue', 'coastal', 'granule_metadata', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 'rededge2', 'rededge3', 'scl', 'swir16', 'swir22', 'thumbnail', 'tileinfo_metadata', 'visual', 'wvp', 'aot-jp2', 'blue-jp2', 'coastal-jp2', 'green-jp2', 'nir-jp2', 'nir08-jp2', 'nir09-jp2', 'red-jp2', 'rededge1-jp2', 'rededge2-jp2', 'rededge3-jp2', 'scl-jp2', 'swir16-jp2', 'swir22-jp2', 'visual-jp2', 'wvp-jp2'])


In [18]:
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

aot: Aerosol optical thickness (AOT)
blue: Blue (band 2) - 10m
coastal: Coastal aerosol (band 1) - 60m
granule_metadata: None
green: Green (band 3) - 10m
nir: NIR 1 (band 8) - 10m
nir08: NIR 2 (band 8A) - 20m
nir09: NIR 3 (band 9) - 60m
red: Red (band 4) - 10m
rededge1: Red edge 1 (band 5) - 20m
rededge2: Red edge 2 (band 6) - 20m
rededge3: Red edge 3 (band 7) - 20m
scl: Scene classification map (SCL)
swir16: SWIR 1 (band 11) - 20m
swir22: SWIR 2 (band 12) - 20m
thumbnail: Thumbnail image
tileinfo_metadata: None
visual: True color image
wvp: Water vapour (WVP)
aot-jp2: Aerosol optical thickness (AOT)
blue-jp2: Blue (band 2) - 10m
coastal-jp2: Coastal aerosol (band 1) - 60m
green-jp2: Green (band 3) - 10m
nir-jp2: NIR 1 (band 8) - 10m
nir08-jp2: NIR 2 (band 8A) - 20m
nir09-jp2: NIR 3 (band 9) - 60m
red-jp2: Red (band 4) - 10m
rededge1-jp2: Red edge 1 (band 5) - 20m
rededge2-jp2: Red edge 2 (band 6) - 20m
rededge3-jp2: Red edge 3 (band 7) - 20m
scl-jp2: Scene classification map (SCL)
swi

In [19]:
assets["thumbnail"].href

'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/35/S/NA/2023/8/S2A_35SNA_20230827_0_L2A/thumbnail.jpg'

![](https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/35/S/NA/2023/8/S2A_35SNA_20230827_0_L2A/thumbnail.jpg)

In [45]:
items[-1].assets['thumbnail'].href

'https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/35/S/NA/2023/7/S2A_35SNA_20230708_0_L2A/thumbnail.jpg'

![](https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/35/S/NA/2023/7/S2A_35SNA_20230708_0_L2A/thumbnail.jpg)

In [21]:
import rioxarray

red_href = assets['red'].href
red = rioxarray.open_rasterio(red_href)
red.rio.to_raster('red.tif', driver='COG')

## Exercise: 

Let's put in practice what we have learned in this episode, and search for the tile of the global [Copernicus Digital Elevation Model (DEM)](https://spacedata.copernicus.eu/collections/copernicus-digital-elevation-model) that includes the island of Rhodes. Use the same STAC endpoint and coordinates as in the previous search, but modify the searched collection (the Copernicus DEM collection has ID: "cop-dem-glo-30"). Extract the URL pointing to the data from the item matching your search. How does it differ from the URL you got for the satellite images?

In [22]:
search = client.search(
    collections=['cop-dem-glo-30'],
    intersects=point,
)

In [ ]:
items = search.item_collection()
items[0].assets['data'].href